In [1]:
import h5py
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import KFold

In [5]:
window_len = 4096   # Window length in samples
hop_perc = 40 # Hop length relative to window size (percents)
hop_len = int((hop_perc / 100.0) * window_len) # Hop length in the STFT calculation
mel_bands_num = 64 # Number of Mel-bands to calculate Mel-band energy feature
signal_len = 20.0 # Signal length in seconds

adj_feat_num = 10 # Number of adjacent features to concatenate (only for 1-D features)
feat_num = mel_bands_num + 3 * (2 * adj_feat_num + 1) # Number of features (mel_bands_num + 1-D time features)
time_samples_num = 539 # Number of time samples where features are calcuated
time_dist_threshold = 0.75 # Time distance threshold

In [18]:
def feature2index(feat_list, feat_inds):
    inds = np.array([])
    for feat in feat_inds:
        if feat in feat_list:
            inds = np.append(inds, feat_inds[feat])
    inds = np.sort(inds).astype('int')
    return inds

def feats_string(feats, feat_inds):
    fs = []
    for feat in feat_inds:
        if feat in feats:
            fs.append(feat)
    fs = "#".join(fs)
    return fs

def compare_prediction_and_gt(y_pred, y_gt, dec_thresh):
    cars_correct = false_pos = false_neg = 0

    for k in range(y_pred.shape[0]):
        gt_row = y_gt[k, :].flatten()
        peaks_gt = find_peaks(np.max(gt_row) - gt_row)[0]
        gt_intervals = np.zeros((peaks_gt.size, 2))
        for j in range(peaks_gt.size):
            aux = peaks_gt[j]
            while aux > 0 and gt_row[aux] < gt_row[aux -1]:
                aux -= 1
            gt_intervals[j, 0] = aux
            aux = peaks_gt[j]
            while aux < gt_row.size -1 and gt_row[aux] < gt_row[aux + 1]:
                aux += 1
            gt_intervals[j, 1] = aux
    
    pred_row = y_pred[k, :].flatten()
    pred_row_inv = np.max(pred_row) - pred_row
    peaks_pos = find_peaks(pred_row_inv, prominence=0.05)[0]
    pred_peak_pos = peaks_pos[np.argwhere(pred_row[peaks_pos] < dec_thresh)].flatten()
    for j in range(pred_peak_pos.size):
        within = np.logcial_and(gt_intervals[:, 0] < pred_peak_pos[j], gt_intervals[:, 1] > pred_peak_pos[j])
        ind_within = np.argwhere(within).flatten()
        if ind_within.size > 0:
            cars_correct += 1
            gt_itervals = np.delete(gt_intervals, ind_within[0], axis=0)
        else:
            flase_pos += 1
    false_neg += gt_intervals.shape[0]
    return cars_correct, false_pos, false_neg

def detection_postprocessing(y_pred):
    y_pred_proc = y_pred.copy()
    for k in range(y_pred_proc.shape[0]):
        row = y_pred_proc[k, :].flatten()
        row = running_mean(row, 7)
        row = running_mean(row, 5)
        row = running_mean(row, 3)
        y_pred_proc[k, :] = row

    return y_pred_proc

def car_detection(y_pred, y_gt, decision_thresh):
    assert y_pred.shape == y_gt.shape, "Predicted and ground truth arrays are not the same dimensions"
    
    y_pred_proc = detection_postprocessing(y_pred)
    correct, false_pos, false_neg = compare_prediction_and_gt(y_pred_proc, y_gt, decision_thresh)

    return correct, false_pos, false_neg


In [6]:
C = 1.0
epsilon = 0.05
feat_inds = {
    "ste" : np.arange(0 * (2 * adj_feat_num + 1), 1 * (2 * adj_feat_num + 1)),
    "trf" : np.arange(1 * (2 * adj_feat_num + 1), 2 * (2 * adj_feat_num + 1)),
    "hfp" : np.arange(2 * (2 * adj_feat_num + 1), 3 * (2 * adj_feat_num + 1)),
    "lms" : np.arange(3 * (2 * adj_feat_num + 1), feat_num)
}
curr_feats = ["ste", "trf", "hfp", "lms"]

In [7]:
dataset = 'datasets/dataset_cross-validation.h5'
hf = h5py.File(dataset, 'r')
x = np.array(hf['features'], dtype=np.float64)
y = np.array(hf['labels'], dtype=np.float64)
vc = np.array(hf['vehicle_count'])
hf.close()

print("X shape: ", x.shape)
print("Y shape: ", y.shape)
print("VC shape: ", vc.shape)

X shape:  (250, 539, 127)
Y shape:  (250, 539)
VC shape:  (250,)


In [12]:
inds_keep = feature2index(curr_feats, feat_inds)
feat_string = feats_string(curr_feats, feat_inds)
x = x[:, :, inds_keep]

Shuffle the data

In [14]:
rand_perm = np.random.RandomState().permutation(x.shape[0])
x = x[rand_perm, :, :]
y = y[rand_perm, :]
vc = vc[rand_perm]

In [16]:
thresh_perc = np.arange(0, 101, 1)
thresh_len = thresh_perc.size
correct = np.zeros((thresh_len,))
false_pos = np.zeros((thresh_len,))
false_neg = np.zeros((thresh_len,))

RVCE = np.zeros((thresh_len,))
y_pred_all = np.zeros_like(y)
y_test_all = np.zeros_like(y)

In [20]:
folds = 5
kf = KFold(n_splits=folds)
kf.get_n_splits(x)
for train_index, test_index in kf.split(x):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train = x_train.reshape(x_train.shape[0] * x_train.shape[1], x_train.shape[2])
    y_train = y_train.reshape(-1, )
    x_test = x_test.reshape(x_test.shape[0] * x_test.shape[1], x_test.shape[2])

    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    regressor = SVR(gamma='scale', C=C, epsilon=epsilon)
    regressor.fit(x_train, y_train)
    y_pred = regressor.predict(x_test)
    y_pred = y_pred.reshape(-1, time_samples_num)
    y_pred_all[test_index, :] = y_pred.copy()
    y_test_all[test_index, :] = y_test.copy()

    veh_count = np.sum(vc[test_index])
    for j in range(thresh_len):
        (corr, fp, fn) = car_detection(y_pred, y_test, (thresh_perc[j] / 100) * time_dist_threshold)
        correct[j] += corr / veh_count
        false_pos[j] += fp / veh_count
        false_neg[j] += fn / veh_count
        RVCE[j] += abs(veh_count - (corr + fp)) / veh_count
    
    print("Fold done")

correct = (correct / folds) * 100
false_pos = (false_pos / folds) * 100
false_neg = (false_neg /folds) * 100
RVCE =(RVCE / folds) * 100

SVR_file = f"SVR_feat#{feat_string}_Exp1.h5"
hf = h5py.File('SVR/' + SVR_file, 'w')
hf.create_dataset('test_pred', data=y_pred_all, compression="gzip")
hf.create_dataset('test_gt', data=y_test_all, compression="gzip")
hf.close()

probs_file = f"Probs_feat#{feat_string}_Exp1.h5"
hf = h5py.File('Probs/' + probs_file, 'w')
hf.create_dataset('correct', data=correct, compression="gzip")
hf.create_dataset('false_pos', data=false_pos, compression="gzip")
hf.create_dataset('false_neg', data=false_neg, compression="gzip")
hf.close()

RVCE_file = f"RVCE_feat#{feat_string}_Exp1.h5"
hf = h5py.File('RVCE/' + RVCE_file, 'w')
hf.create_dataset('RVCE', data=RVCE, compression="gzip")
hf.close()
